In [2]:
import networkx as nx
from gurobipy import *

In [3]:
def csvToList(filename):
    with open(filename,'r') as f:
        # allLine : CSV的每row變成一string的string list 
        # readlines後面的[1:]表示從第一行之後開始讀，也可以用next(f)達到這個目的
        allLines = f.readlines()[1:]  
        f.closed
        dataList = [] 
        for line in allLines: # line : CSV各row變成的string
            # line.split(',')將各line以','為單位分割，x.strip()是刪去分割後各element的前後空白符和換行符號
            item = [x.strip() for x in line.split(',')] 
            item = list(map(float, item))
            # 在dataList這個二維list中加入item這個list
            dataList.append(item) 
    return dataList    

def loadDis(nodeNum, dataList):
    dis = {}
    for i in range(nodeNum):
        for j in range(nodeNum):
            dis[i,j] = abs(dataList[i][1]-dataList[j][1]) + abs(dataList[i][2]-dataList[j][2])
    return dis    
    
def loadNetSupply(nodeNum, dataList):
    netSupply = {}
    for i in range(nodeNum):
        netSupply[i] = float(dataList[i][3])
    return netSupply

In [4]:
m = Model("mip1")
nodeList = csvToList('20node_1.csv')
nodeNum = len(nodeList)

dis = loadDis(nodeNum, nodeList)
netSupply = loadNetSupply(nodeNum, nodeList)
upperB = 70
cost = 8

flow = {}
x = {}
for i in range(nodeNum):
    for j in range(nodeNum):
        flow[i,j] = m.addVar(vtype=GRB.INTEGER, name = 'flow_%s_%s'%(i,j))
        x[i,j] = m.addVar(vtype=GRB.BINARY, name = 'x_%s_%s'%(i,j))
m.update()

for i in range(nodeNum):
    m.addConstr(quicksum(flow[i,j] for j in range(nodeNum))
                         -quicksum(flow[j,i] for j in range(nodeNum)) == netSupply[i], 'flowBal_%s'%(i))
for i in range(nodeNum):
    for j in range(nodeNum):
        m.addConstr(flow[i,j] <= x[i,j]*upperB, 'upper_%s_%s'%(i, j))

for i in range(nodeNum):
    m.addConstr(quicksum(x[i,j] for j in range(nodeNum)) <= 1, 'oneOuput_%s'%(i))
        
m.setObjective(quicksum(x[i,j]*cost*dis[i,j]/10 for i in range(nodeNum) for j in range(nodeNum)), GRB.MINIMIZE)

m.optimize()



Using license file C:\Users\User\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 840 rows, 1568 columns and 3864 nonzeros
Model fingerprint: 0xc8f9a485
Variable types: 0 continuous, 1568 integer (784 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 872.0000000
Presolve removed 36 rows and 64 columns
Presolve time: 0.01s
Presolved: 804 rows, 1504 columns, 3764 nonzeros
Variable types: 0 continuous, 1504 integer (748 binary)

Root relaxation: objective 1.636571e+01, 1070 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   16.36571    0   19  872.00000   16.36571  98.1%     -    0s
H    0     0                    

In [5]:
if m.status == GRB.Status.OPTIMAL:
    G = nx.DiGraph()
    for i in range(nodeNum):
        G.add_node(i, pos=(nodeList[i][1],nodeList[i][2]), netSupply = netSupply[i]/10)
        for j in range(nodeNum):
            if x[i,j].x > 0:
                #print (i, '->', j, ':', flow[i,j].x)
                G.add_edge(i, j, weight=flow[i,j])
    
    nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True)  
    #nx.draw(G, nx.get_node_attributes(G, 'pos'))
    nx.draw_networkx_edges(G, nx.get_node_attributes(G, 'pos'), width=1.0, alpha=0.5)
    #nx.draw_networkx_edge_labels(G, nx.get_node_attributes(G, 'pos'), labels = nx.get_edge_attributes(G,'f'), font_size=7.5)
    nx.draw_networkx_nodes(G, nx.get_node_attributes(G, 'pos'), G.nodes(), node_color="#98e8c5")
    nx.draw_networkx_labels(G, nx.get_node_attributes(G, 'pos'), labels = nx.get_node_attributes(G,'netSupply'), font_color='r')
            

C:\Users\User\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):
C:\Users\User\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:676: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if cb.iterable(node_size):  # many node sizes
